# Exploratory Analysis
This notebook performs exploratory data analysis on the financial data used for forecasting.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from src.data_loader import fetch_data

# Fetch data for S&P500
data = fetch_data('^GSPC', '2010-01-01', '2023-01-01')
data.plot(title='S&P500 Adjusted Close Price')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()